In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import shutil
import logging
import subprocess
from copy import deepcopy
from time import time
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.rdDistGeom import EmbedMultipleConfs
from linux_qm.src.util import _load_smiles3D, _create_tmp_dir, draw3Dconfs, SetPositions
from linux_qm.qm.crest.crest import conformer_pipeline


In [3]:
smi = 'COC1CN(C)C1'
# smi = 'CO'


# mol = _load_smiles3D(smi)
# EmbedMultipleConfs(mol, numConfs=3)

mol = conformer_pipeline(smi)

# Get the conformers

conformers = mol.GetConformers()

# Print the number of conformers generated
print(f"Number of conformers generated: {len(conformers)}")
print("Energies:", [c.GetProp('energy') for c in conformers])

Number of conformers generated: 4
Energies: ['-23.275472740000001', '-23.273513529999999', '-23.273084870000002', '-23.27281279']


In [4]:
query = Chem.MolFromSmarts('C1CCN1')
atom_ids = mol.GetSubstructMatch(query)
print(atom_ids)
AllChem.AlignMolConformers(mol, atomIds=atom_ids)
draw3Dconfs(mol, autoalign=False)

(3, 2, 6, 4)
num conformers 4


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [5]:
import os
os.getcwd()

'/home/ergot/linux_qm/notebooks'

In [6]:
logging.getLogger().setLevel(logging.WARNING)

In [7]:
from linux_qm.qm.orca.orca import OrcaDriver
orca = OrcaDriver()

conf = deepcopy(mol).GetConformer()

orca.options['method'] = 'HF def2-SVP'
orca.single_point(conf)

[24-11-2023 02:31:29] [DEBUG] ORCA INPUT:
!HF def2-SVP   
%geom MaxIter 100 end
%pal nprocs 16 end
%maxcore 2000
*xyz 0 1
C            2.71482748          0.42882047         -0.15403943
O            1.61469165         -0.39693456         -0.44575399
C            0.56782242         -0.27293135          0.47464749
C           -0.66124890         -1.11990884          0.13117785
N           -1.51499794          0.04689679          0.36976590
C           -2.58126784          0.29502558         -0.56444822
C           -0.36188608          0.95004635          0.33951359
H            3.12947485          0.19816329          0.83510407
H            2.43975670          1.48949190         -0.18101609
H            3.46304260          0.22804311         -0.91807065
H            0.93021521         -0.41949934          1.50069183
H           -0.88481317         -1.96630143          0.78045724
H           -0.62918750         -1.44665393         -0.91403550
H           -2.23234852          0.34931098   

In [8]:
orca.geometry_optimization(conf)

[24-11-2023 02:31:33] [INFO] Geometry optimization
[24-11-2023 02:31:33] [INFO] Method: HF def2-SVP
[24-11-2023 02:31:33] [DEBUG] ORCA INPUT:
!HF def2-SVP OPT  
%geom MaxIter 100 end
%pal nprocs 16 end
%maxcore 2000
*xyz 0 1
C            2.71482748          0.42882047         -0.15403943
O            1.61469165         -0.39693456         -0.44575399
C            0.56782242         -0.27293135          0.47464749
C           -0.66124890         -1.11990884          0.13117785
N           -1.51499794          0.04689679          0.36976590
C           -2.58126784          0.29502558         -0.56444822
C           -0.36188608          0.95004635          0.33951359
H            3.12947485          0.19816329          0.83510407
H            2.43975670          1.48949190         -0.18101609
H            3.46304260          0.22804311         -0.91807065
H            0.93021521         -0.41949934          1.50069183
H           -0.88481317         -1.96630143          0.78045724
H      

In [9]:
%%time

logging.getLogger().setLevel(logging.INFO)
orca.options['solvent'] = 'THF'

conf = deepcopy(mol).GetConformer()

orca.options['method'] = 'M062X 6-311++G(d,p) NMR'
orca.geometry_optimization(conf)

[24-11-2023 02:32:03] [INFO] Geometry optimization
[24-11-2023 02:32:03] [INFO] Method: M062X 6-311++G(d,p) NMR
[24-11-2023 02:34:15] [INFO] Identified logfile to be in ORCA format
[24-11-2023 02:34:15] [INFO] Success: True
[24-11-2023 02:34:15] [INFO] Num Iter: 9
[24-11-2023 02:34:15] [INFO] Elapsed Time: 131.8s


CPU times: user 126 ms, sys: 61.7 ms, total: 187 ms
Wall time: 2min 11s


In [10]:
%%time

conf = deepcopy(mol).GetConformer()

orca.options['method'] = 'M062X 6-31G'
orca.geometry_optimization(conf)

orca.options['method'] = 'M062X 6-311++G(d,p) NMR'
orca.geometry_optimization(conf)

[24-11-2023 02:34:15] [INFO] Geometry optimization
[24-11-2023 02:34:15] [INFO] Method: M062X 6-31G
[24-11-2023 02:35:07] [INFO] Identified logfile to be in ORCA format
[24-11-2023 02:35:07] [INFO] Success: True
[24-11-2023 02:35:07] [INFO] Num Iter: 9
[24-11-2023 02:35:07] [INFO] Elapsed Time: 52.0s
[24-11-2023 02:35:07] [INFO] Geometry optimization
[24-11-2023 02:35:07] [INFO] Method: M062X 6-311++G(d,p) NMR
[24-11-2023 02:36:48] [INFO] Identified logfile to be in ORCA format
[24-11-2023 02:36:48] [INFO] Success: True
[24-11-2023 02:36:48] [INFO] Num Iter: 7
[24-11-2023 02:36:48] [INFO] Elapsed Time: 101.1s


CPU times: user 243 ms, sys: 48.6 ms, total: 291 ms
Wall time: 2min 33s


In [11]:
%%time

conf = deepcopy(mol).GetConformer()
orca.options['method'] = 'M062X cc-pVDZ NMR'
orca.geometry_optimization(conf)

[24-11-2023 02:36:48] [INFO] Geometry optimization
[24-11-2023 02:36:48] [INFO] Method: M062X cc-pVDZ NMR
[24-11-2023 02:38:10] [INFO] Identified logfile to be in ORCA format
[24-11-2023 02:38:10] [INFO] Success: True
[24-11-2023 02:38:10] [INFO] Num Iter: 9
[24-11-2023 02:38:10] [INFO] Elapsed Time: 82.3s


CPU times: user 127 ms, sys: 48.6 ms, total: 175 ms
Wall time: 1min 22s


In [12]:
%%time
conf = deepcopy(mol).GetConformer()
orca.options['method'] = 'M062X cc-pVTZ NMR'
orca.geometry_optimization(conf)

[24-11-2023 02:38:10] [INFO] Geometry optimization
[24-11-2023 02:38:10] [INFO] Method: M062X cc-pVTZ NMR
[24-11-2023 02:41:28] [INFO] Identified logfile to be in ORCA format
[24-11-2023 02:41:28] [INFO] Success: True
[24-11-2023 02:41:28] [INFO] Num Iter: 9
[24-11-2023 02:41:28] [INFO] Elapsed Time: 198.2s


CPU times: user 175 ms, sys: 49.4 ms, total: 225 ms
Wall time: 3min 18s


In [13]:
%%time
conf = deepcopy(mol).GetConformer()
orca.options['method'] = 'M062X aug-cc-pVTZ NMR'
orca.geometry_optimization(conf)

[24-11-2023 02:41:28] [INFO] Geometry optimization
[24-11-2023 02:41:28] [INFO] Method: M062X aug-cc-pVTZ NMR
[24-11-2023 02:50:45] [INFO] Identified logfile to be in ORCA format
[24-11-2023 02:50:45] [INFO] Success: True
[24-11-2023 02:50:45] [INFO] Num Iter: 9
[24-11-2023 02:50:45] [INFO] Elapsed Time: 556.3s


CPU times: user 195 ms, sys: 80.9 ms, total: 276 ms
Wall time: 9min 16s


In [17]:
%%time

conf = deepcopy(mol).GetConformer()
orca.options['method'] = 'M062X cc-pVDZ'
orca.geometry_optimization(conf)

orca.options['method'] = 'M062X aug-cc-pVTZ NMR'
orca.geometry_optimization(conf)

[24-11-2023 03:03:21] [INFO] Geometry optimization
[24-11-2023 03:03:21] [INFO] Method: M062X cc-pVDZ
[24-11-2023 03:04:42] [INFO] Identified logfile to be in ORCA format
[24-11-2023 03:04:42] [INFO] Success: True
[24-11-2023 03:04:42] [INFO] Num Iter: 9
[24-11-2023 03:04:42] [INFO] Elapsed Time: 80.5s
[24-11-2023 03:04:42] [INFO] Geometry optimization
[24-11-2023 03:04:42] [INFO] Method: M062X aug-cc-pVTZ NMR
[24-11-2023 03:10:10] [INFO] Identified logfile to be in ORCA format
[24-11-2023 03:10:10] [INFO] Success: True
[24-11-2023 03:10:10] [INFO] Num Iter: 5
[24-11-2023 03:10:10] [INFO] Elapsed Time: 328.7s


CPU times: user 234 ms, sys: 122 ms, total: 356 ms
Wall time: 6min 49s


In [15]:
query = Chem.MolFromSmarts('C1CCN1')
atom_ids = mol.GetSubstructMatch(query)
print(atom_ids)
AllChem.AlignMolConformers(mol, atomIds=atom_ids)
draw3Dconfs(mol, confIds=[0], autoalign=False)

(3, 2, 6, 4)
num conformers 4


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [16]:
data.scfvalues

NameError: name 'data' is not defined

In [ ]:
data.scftargets

In [ ]:
# NMR shielding
for k,v in data.nmrtensors.items():
    print(mol.GetAtomWithIdx(k).GetSymbol(), k, v['isotropic'])

In [ ]:
dir(data)

In [ ]:
print(data.metadata['cpu_time'][0])

In [ ]:
len(data.converged_geometries)

In [ ]:
dir(data)

In [ ]:
data.optdone

In [ ]:
data.metadata

In [ ]:
data.new_geometries

In [ ]:
# HOMO LUMO
homo = data.moenergies[0][data.homos[0]]
lumo = data.moenergies[0][data.homos[0] + 1]
homo, lumo

In [ ]:
print(data.metadata['cpu_time'][0])

In [ ]:
data.natom, data.nmo

In [ ]:
data.metadata

In [ ]:
data.scfenergies